In [22]:
%load_ext autoreload
%autoreload 2
import os
import glob
from pysentimiento.utils import load_evaluation

base_path = "../../../evaluations/lince/ner/dev"

evaluation_paths = glob.glob(f"{base_path}/*.json")
print(len(evaluation_paths))

models = {}
results = []

for path in evaluation_paths:
    model_evaluation = load_evaluation(path)
    model_name = model_evaluation["model"].split("/")[-1]

    print("="*50)
    print(model_name)
    for task, task_evaluations in model_evaluation["evaluations"].items():
        print(task)
        print(len(task_evaluations))
        for run in task_evaluations:
            results.append({
                **{
                    "model": model_name,
                    "task": task,
                },
                **run,
            })



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
5
xlm-roberta-base
ner
2
bert-base-multilingual-uncased
ner
2
robertuito-base-uncased
ner
4
robertuito-base-deacc
ner
3
robertuito-base-cased
ner
2


In [23]:
import pandas as pd

pd.options.display.float_format = '{:.3f}'.format

mean_df = pd.DataFrame(results).groupby(["model"]).mean()
std_df = pd.DataFrame(results).groupby(["model"]).std()
# Magia negra

#mean_df = mean_df.unstack(1)
#std_df = std_df.unstack(1)

cols = ["micro_f1", "macro_f1", "accuracy", "precision", "recall"]
other_f1_cols = [c for c in mean_df.columns if "f1" in c and c not in cols]

idx = mean_df.index

mean_df.loc[idx, cols].round(3).astype(str) + " ± " + std_df.loc[idx, cols].round(3).astype(str)

#mean_df.loc[idx, cols].sort_values(by=["micro_f1"], ascending=False)

,micro_f1,macro_f1,accuracy,precision,recall
model,,,,,
bert-base-multilingual-uncased,0.647 ± 0.005,0.506 ± 0.003,0.985 ± 0.001,0.635 ± 0.031,0.66 ± 0.024
robertuito-base-cased,0.666 ± 0.005,0.51 ± 0.005,0.986 ± 0.001,0.663 ± 0.024,0.67 ± 0.014
robertuito-base-deacc,0.674 ± 0.004,0.532 ± 0.003,0.986 ± 0.0,0.668 ± 0.004,0.68 ± 0.003
robertuito-base-uncased,0.672 ± 0.003,0.509 ± 0.011,0.986 ± 0.0,0.673 ± 0.006,0.67 ± 0.011
xlm-roberta-base,0.64 ± 0.001,0.5 ± 0.008,0.985 ± 0.0,0.641 ± 0.007,0.638 ± 0.005


In [24]:
pd.options.display.max_columns = None
mean_df.loc[idx, cols + other_f1_cols].sort_values(by=["micro_f1"], ascending=False)

,micro_f1,macro_f1,accuracy,precision,recall,EVENT_f1,GROUP_f1,LOC_f1,ORG_f1,OTHER_f1,PER_f1,PROD_f1,TIME_f1,TITLE_f1
model,,,,,,,,,,,,,,
robertuito-base-deacc,0.674,0.532,0.986,0.668,0.680,0.356,0.598,0.741,0.442,0.388,0.805,0.552,0.456,0.453
robertuito-base-uncased,0.672,0.509,0.986,0.673,0.670,0.335,0.559,0.746,0.419,0.312,0.815,0.522,0.414,0.457
robertuito-base-cased,0.666,0.510,0.986,0.663,0.670,0.355,0.571,0.736,0.390,0.405,0.807,0.544,0.372,0.412
bert-base-multilingual-uncased,0.647,0.506,0.985,0.635,0.660,0.301,0.547,0.726,0.467,0.379,0.785,0.505,0.448,0.401
xlm-roberta-base,0.640,0.500,0.985,0.641,0.638,0.379,0.460,0.720,0.425,0.365,0.778,0.532,0.435,0.406
